In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
import copy
import random
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
nltk.download('punkt')
import json
%matplotlib inline

import warnings
warnings.filterwarnings(action="ignore")

pd.set_option('display.max_rows', None)
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/elicer/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/elicer/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [2]:
# 시드 고정
import os

SEED=42
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)

In [3]:
train = pd.read_csv("train.csv") # 학습용 데이터
test = pd.read_csv("submission.csv") # 테스트 데이터(제출파일의 데이터)

In [4]:
# 학습 타겟 데이터
targets=train['is_converted']
rows=train.shape[0]

with open('permitted.json','r') as f:
    permitted=json.load(f)

# 
train=train.drop('is_converted',axis=1)
test=test.drop('id',axis=1)
test=test.drop('is_converted',axis=1)

In [5]:
# columns
cols_by_type={}

cols_by_type['categorical']=train.columns[train.dtypes=='object'].tolist()
cols_by_type['numerical']=train.columns[train.dtypes!='object'].tolist()
#colsByType['numerical'].remove('is_converted')

print('\nnumerical columns: '+str(len(cols_by_type['numerical'])))
print('categorical columns: '+str(len(cols_by_type['categorical'])))
print('total columns: '+str(len(cols_by_type['numerical'])+len(cols_by_type['categorical'])))


numerical columns: 13
categorical columns: 15
total columns: 28


In [6]:
# delete cols
del_cols=['ver_win_rate_x','ver_win_ratio_per_bu',
          'business_subarea', 'product_subcategory', 'product_modelname', 
          'customer_country.1']

# preserve
# preserve=pd.DataFrame()
# preserve['com_reg_ver_win_rate']=total_data['com_reg_ver_win_rate']

train_process=train.drop(del_cols,axis=1)
test_process=test.drop(del_cols,axis=1)

In [7]:
# id_strategic_ver it_strategic_ver idit_strategic_ver
ver=['id_strategic_ver', 'it_strategic_ver', 'idit_strategic_ver']
train_process['strategic_ver']=np.where(train_process['idit_strategic_ver']>0,1,0)
test_process['strategic_ver']=np.where(test_process['idit_strategic_ver']>0,1,0)
train_process=train_process.drop(ver,axis=1)
test_process=test_process.drop(ver,axis=1)

In [8]:
# country columns
# region
def preprocess_region(x,permitted):
    if type(x)==type(''):
        if permitted.get(x):
            return permitted[x]
        return 'OT'
    return np.nan

def response_corporate_encoding(train_data):
    permit={}
    
    for train_label in train_data.value_counts().index:
        permit[train_label]=1
    permit['OT']=1

    return permit

def preprocess_response_corporate(x,permitted):
    if type(x)==type(''):
        if permitted.get(x):
            return x
        return 'OT'
    return np.nan


def country_encoding(train_data):
    permit={}

    train_labels=train_data.apply(lambda x:x.lower().replace(' ','').replace('/',' ').split(' ')[-1] if type(x)==type('') else np.nan).value_counts()
    train_labels=sorted(train_labels.items(),key=lambda x:x[1],reverse=True)

    for train_label in train_labels[:100]:
        if train_label[0]:
            permit[f'{train_label[0]}']=1
    
    if permit.get(''):
        del permit['']
    
    # 
    permit['OT']='OT'
    permit['dump_key']='dump_value'
    permit['türkiye']='turkey'
    del permit['700patrooncreekblvdalbanyny12206']
    del permit['uaedubai']
    
    return permit

def preprocess_customer_country(x,permitted):
    if type(x)==type(''):
        x=x.lower().replace(' ','').replace('/',' ')
        for word in x.split(' '):
            if permitted.get(word):
                return word
        return 'OT'
    return np.nan

# region
train_process['region']=train_process['response_corporate'].apply(lambda x:preprocess_region(x,permitted=permitted['region']))
test_process['region']=test_process['response_corporate'].apply(lambda x:preprocess_region(x,permitted=permitted['region']))

# response_corporate
permitted['response_corporate']=response_corporate_encoding(train['response_corporate'])
train_process['response_corporate']=train_process['response_corporate'].apply(lambda x:preprocess_response_corporate(x,permitted=permitted['response_corporate']))
test_process['response_corporate']=test_process['response_corporate'].apply(lambda x:preprocess_response_corporate(x,permitted=permitted['response_corporate']))

# customer_country   
permitted['customer_country']=country_encoding(train['customer_country'])
train_process['customer_country']=train_process['customer_country'].apply(lambda x:preprocess_customer_country(x,permitted=permitted['customer_country']))
test_process['customer_country']=test_process['customer_country'].apply(lambda x:preprocess_customer_country(x,permitted=permitted['customer_country']))

In [9]:
# business_unit
train_process['business_unit']=train_process['business_unit'].replace('Solution','ETC')
train_process['business_unit']=train_process['business_unit'].replace('CM','ETC')

test_process['business_unit']=test_process['business_unit'].replace('Solution','ETC')
test_process['business_unit']=test_process['business_unit'].replace('CM','ETC')

In [10]:
# customer_type
def preprocess_customer_type(x,permitted):
    if type(x)==type(''):
        x=x.lower().replace('-','').replace(' ','')
        if permitted.get(x):
            return permitted[x]
        else:
            return 'OT'
    return x
    
train_process['customer_type']=train_process['customer_type'].apply(lambda x:preprocess_customer_type(x,permitted=permitted['customer_type']))
test_process['customer_type']=test_process['customer_type'].apply(lambda x:preprocess_customer_type(x,permitted=permitted['customer_type']))

In [11]:
# business_area
# total_data['business_area']=total_data['business_area'].replace('hospital & health care','ETC')
# total_data['business_area']=total_data['business_area'].replace('factory','ETC')
# total_data['business_area']=total_data['business_area'].replace('government department','ETC')
# total_data['business_area']=total_data['business_area'].replace('public facility','ETC')
# total_data['business_area']=total_data['business_area'].replace('transportation','ETC')
# total_data['business_area']=total_data['business_area'].replace('power plant / renewable energy','ETC')
train_process['business_area']=train_process['business_area'].fillna('UNK')
test_process['business_area']=test_process['business_area'].fillna('UNK')

In [12]:
# ver_cus, ver_pro
grant=['ver_cus', 'ver_pro']
train_process['grant_weight']=np.where(train_process['ver_cus']>0,1,0)
train_process['grant_weight']=np.where(train_process['ver_pro']>0,1,train_process['grant_weight'])
train_process=train_process.drop(grant,axis=1)

test_process['grant_weight']=np.where(test_process['ver_cus']>0,1,0)
test_process['grant_weight']=np.where(test_process['ver_pro']>0,1,test_process['grant_weight'])
test_process=test_process.drop(grant,axis=1)

In [13]:
# expected_timeline
def preprocess_expected_timeline(x,permitted):
    if type(x)==type(''):
        x=x.lower().replace(' ','').replace('_','')
        if permitted.get(x):
            return permitted[x]
        return 'OT'
    return x

train_process['expected_timeline']=train_process['expected_timeline'].apply(lambda x:preprocess_expected_timeline(x,permitted=permitted['expected_timeline']))
test_process['expected_timeline']=test_process['expected_timeline'].apply(lambda x:preprocess_expected_timeline(x,permitted=permitted['expected_timeline']))

In [14]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

# lead_desc_length, historical_existing_cnt
numerical=['lead_desc_length','historical_existing_cnt']
#total_data[numerical]=scaler.fit_transform(total_data[numerical])
train_process[numerical]=np.log1p(train_process[numerical])
test_process[numerical]=np.log1p(test_process[numerical])

In [15]:
# inquiry_type
def preprocess_inquiry_type(x,permitted):
    if type(x)==type(''):
        x=x.lower().replace('_',' ')
        for word in x.split(' '):
            if permitted.get(word):
                return permitted[word]
        return 'OT'
    return np.nan

train_process['inquiry_type']=train_process['inquiry_type'].apply(lambda x:preprocess_inquiry_type(x,permitted=permitted['inquiry_type']))
test_process['inquiry_type']=test_process['inquiry_type'].apply(lambda x:preprocess_inquiry_type(x,permitted=permitted['inquiry_type']))

In [16]:
# customer_job
def preprocess_customer_job(x,permitted):
    if type(x)==type(''):
        porter=PorterStemmer()
        tokens=word_tokenize(x)
        stems=[porter.stem(token) for token in tokens]
        for stem in stems:
            if permitted.get(stem):
                return permitted[stem]
        return 'OT'
    return np.nan

train_process['customer_job']=train_process['customer_job'].apply(lambda x:preprocess_customer_job(x,permitted=permitted['customer_job']))
test_process['customer_job']=test_process['customer_job'].apply(lambda x:preprocess_customer_job(x,permitted=permitted['customer_job']))

In [17]:
# product_category
def preprocess_product_category(x,permitted):
    if type(x)==type(''):
        porter=PorterStemmer()
        tokens=word_tokenize(x)
        stems=[porter.stem(token) for token in tokens]

        prefer={}
        for pf in permitted['dump_key']:
            prefer[pf]=0
        
        for stem in stems:
            if permitted.get(stem):
                prefer[permitted[stem]]=1

        for pf in permitted['dump_key']:
            if prefer[pf]>0:
                return pf
        return 'OT'
    return np.nan

train_process['product_category']=train_process['product_category'].apply(lambda x:preprocess_product_category(x,permitted=permitted['product_category']))
test_process['product_category']=test_process['product_category'].apply(lambda x:preprocess_product_category(x,permitted=permitted['product_category']))

In [18]:
# customer_poisition
def preprocess_customer_position(x,permitted):
    if type(x)==type(''):
        x=x.lower().replace('-',' ').replace('/',' ')
        for word in x.split(' '):
            if permitted.get(word):
                return permitted[word]
        return 'OT'
    return np.nan

train_process['customer_position']=train_process['customer_position'].apply(lambda x:preprocess_customer_position(x,permitted=permitted['customer_position']))
test_process['customer_position']=test_process['customer_position'].apply(lambda x:preprocess_customer_position(x,permitted=permitted['customer_position']))

In [19]:
# one hot encoding columns
origin_data=train_process.drop('com_reg_ver_win_rate',axis=1)
origin_columns=origin_data.columns.to_list()
object_columns=origin_data.columns.to_list()
object_columns.remove('bant_submit')
object_columns.remove('historical_existing_cnt')
object_columns.remove('lead_desc_length')
object_columns.remove('strategic_ver')
object_columns.remove('grant_weight')
object_columns.remove('customer_idx')
object_columns.remove('lead_owner')
for col in object_columns:
    permitted['values'][col]=train_process[col].value_counts().index
permitted['values']

{'customer_country': Index(['india', 'brazil', 'unitedstates', 'OT', 'mexico', 'philippines',
        'colombia', 'u.a.e', 'unitedkingdom', 'saudiarabia', 'chile', 'italy',
        'peru', 'germany', 'poland', 'egypt', 'vietnam', 'spain', 'argentina',
        'hongkong', 'australia', 'panama', 'france', 'canada', 'turkey',
        'ecuador', 'indonesia', 'türkiye', 'singapore', 'southafrica', 'iraq',
        'nigeria', 'thailand', 'hungary', 'portugal', 'kenya', 'malaysia',
        'bulgaria', 'costarica', 'dominicanrepublic', 'israel', 'oman',
        'elsalvador', 'pakistan', 'guatemala', 'kuwait', 'bangladesh', 'qatar',
        'switzerland', 'china', 'bolivia', 'honduras', 'lebanon', 'taiwan',
        'netherlands', 'belgium', 'bahrain', 'venezuela', 'puertorico',
        'greece', 'japan', 'afghanistan', 'algeria', 'morocco', 'romania',
        'ghana', 'jordan', 'croatia', 'nicaragua', 'ireland', 'maldives',
        'serbia', 'srilanka', 'uruguay', 'albania', 'jamaica', 'southkor

In [20]:
# encoder
class Encoder():
    def __init__(self):
        self.classes=[]

    def fit(self,data):
        for value in data.value_counts().index:
            self.classes.append(value)

    def transform(self,data):
        result=data.copy(deep=True)
        for i,value in enumerate(self.classes):
            result=result.replace(value,i)
        return result
    
    def inverse_transform(self,data):
        result=data.copy(deep=True)
        for i in range(0,len(self.classes)):
            result=result.replace(i,self.classes[i])
        return result

In [21]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor

imputer=IterativeImputer(estimator=RandomForestRegressor(random_state=SEED),initial_strategy='most_frequent',max_iter=10,random_state=SEED,skip_complete=False,verbose=1)

# encoder
encoders={}
for object_column in object_columns:
    encoders[object_column]=Encoder()
    encoders[object_column].fit(train_process[object_column])


def order_encoding(target_data,object_columns,encoders):
    result=pd.DataFrame()
    for col in target_data.columns:
        if col in object_columns:
            result[col]=encoders[col].transform(target_data[col])
        else:
            result[col]=target_data[col]

    return result

def order_decoding(target_data,object_columns,encoders):
    result=pd.DataFrame()
    for col in target_data.columns:
        if col in object_columns:
            result[col]=target_data[col].apply(lambda x:round(x)).astype(int)
            result[col]=encoders[col].inverse_transform(result[col])
        else:
            result[col]=target_data[col]
    return result

# train data
train_dummy=order_encoding(train_process,object_columns,encoders)
imputer.fit(train_dummy)  
train_dummy_imputed=pd.DataFrame(data=imputer.transform(train_dummy),columns=train_process.columns)
train_imputed=order_decoding(train_dummy_imputed,object_columns,encoders)

# test data
test_dummy=order_encoding(test_process,object_columns,encoders)
test_dummy_imputed=pd.DataFrame(data=imputer.transform(test_dummy),columns=test_process.columns)
test_imputed=order_decoding(test_dummy_imputed,object_columns,encoders)


[IterativeImputer] Completing matrix with shape (59299, 20)
[IterativeImputer] Change: 84.70378337385961, scaled tolerance: 47.466 
[IterativeImputer] Change: 35.66999999999999, scaled tolerance: 47.466 
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (59299, 20)
[IterativeImputer] Completing matrix with shape (5271, 20)


In [22]:
# data kfold
from sklearn.model_selection import KFold

train_datas=[]
train_imputed['is_converted']=targets
train_data_false=train_imputed[train_imputed['is_converted']==0]
train_data_true=train_imputed[train_imputed['is_converted']==1]

# kfold
K=11
dkf=KFold(n_splits=K,shuffle=True,random_state=SEED)
preserves=[pd.DataFrame() for x in range(0,K+1)]
for i,(_,index) in enumerate(dkf.split(train_data_false)):
    print(f'-{i+1} fold data-')
    y_data_false=train_data_false['is_converted'].iloc[index]
    X_data_false=train_data_false.drop('is_converted',axis=1).iloc[index]
    
    y_data=pd.concat([y_data_false,train_data_true['is_converted']],ignore_index=True)
    X_data=pd.concat([X_data_false,train_data_true.drop('is_converted',axis=1)],ignore_index=True)

    # preserve
    # preserves[i]['com_reg_ver_win_rate']=X_data['com_reg_ver_win_rate']
    # X_data=X_data.drop('com_reg_ver_win_rate',axis=1)
    # preserves[i]['customer_idx']=X_data['customer_idx']
    # X_data['customer_idx']=np.log1p(X_data['customer_idx'])
    # preserves[i]['lead_owner']=X_data['lead_owner']
    # X_data['lead_owner']=np.log1p(X_data['lead_owner'])
    

    print(f'X data shape: {X_data.shape}')
    print(f'y data shape: {y_data.shape}')
    train_datas.append((X_data,y_data))

# preserve
# preserves[K]['com_reg_ver_win_rate']=test_process['com_reg_ver_win_rate']
# test_data=test_process.drop('com_reg_ver_win_rate',axis=1)
# preserves[K]['customer_idx']=X_data['customer_idx']
# test_data['customer_idx']=np.log1p(test_data['customer_idx'])
# preserves[K]['lead_owner']=X_data['lead_owner']
# test_data['lead_owner']=np.log1p(test_data['lead_owner'])
test_data=test_imputed
print(f'test data shape: {test_data.shape}')

-1 fold data-
X data shape: (9800, 20)
y data shape: (9800,)
-2 fold data-
X data shape: (9800, 20)
y data shape: (9800,)
-3 fold data-
X data shape: (9800, 20)
y data shape: (9800,)
-4 fold data-
X data shape: (9800, 20)
y data shape: (9800,)
-5 fold data-
X data shape: (9800, 20)
y data shape: (9800,)
-6 fold data-
X data shape: (9800, 20)
y data shape: (9800,)
-7 fold data-
X data shape: (9800, 20)
y data shape: (9800,)
-8 fold data-
X data shape: (9800, 20)
y data shape: (9800,)
-9 fold data-
X data shape: (9800, 20)
y data shape: (9800,)
-10 fold data-
X data shape: (9800, 20)
y data shape: (9800,)
-11 fold data-
X data shape: (9799, 20)
y data shape: (9799,)
test data shape: (5271, 20)


In [23]:
# process data type
for (train_data,target) in train_datas:
    train_data['customer_idx']=train_data['customer_idx'].astype(str)
    train_data['lead_owner']=train_data['lead_owner'].astype(str)
    train_data['strategic_ver']=train_data['strategic_ver'].astype(int)
    train_data['grant_weight']=train_data['grant_weight'].astype(int)
    target=target.apply(lambda x:1 if x else 0)

test_data['customer_idx']=test_data['customer_idx'].astype(str)
test_data['lead_owner']=test_data['lead_owner'].astype(str)
test_data['strategic_ver']=test_data['strategic_ver'].astype(int)
test_data['grant_weight']=test_data['grant_weight'].astype(int)

In [24]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
!pip install catboost
from catboost import CatBoostClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics         import mean_squared_error

In [25]:
# 모델 성능 테스트
def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))
    return F1

In [26]:
class KMODEL:
    def __init__(self,dataset_K,train_K=5):
        self.k_data=dataset_K
        self.k_fold=train_K
        self.models=[[] for i in range(0,K)]
        self.scores=[[] for i in range(0,K)]
        self.thresholds=[[] for i in range(0,K)]
        self.cv_scores=[]

    def modeling_kfold(self,iters,n_estimators,max_depth,learning_rate,cat_features,train_data,targets_data,core):
        # k-fold
        kf=StratifiedKFold(n_splits=self.k_fold,shuffle=True,random_state=SEED)

        for i,(train_index,val_index) in enumerate(kf.split(train_data,targets_data)):
            print(f'-[{iters+1}-{i+1}] fold-')
            X_train,X_val=train_data.iloc[train_index],train_data.iloc[val_index]
            y_train,y_val=targets_data.iloc[train_index],targets_data.iloc[val_index]

            # logloss
            classifier=CatBoostClassifier(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, eval_metric='F1', random_state=SEED, bootstrap_type ='Bernoulli',task_type=core) # default:logloss
            
            # randomforst
            #classifier=RandomForestClassifier(n_estimators=n_estimators,max_depth=max_depth,random_state=SEED)

            # regressor=CatBoostRegressor(n_estimators=n_estimators, max_depth=max_depth, learning_rate=learning_rate, eval_metric='RMSE',random_state=SEED, bootstrap_type ='Bernoulli',task_type=core)

            model=classifier.fit(X_train, y_train, eval_set=(X_val,y_val),verbose=100, early_stopping_rounds=100,cat_features=cat_features,use_best_model=True)
            #model=classifier.fit(X_train,y_train)
            # model=regressor.fit(X_train, y_train, eval_set=(X_val,y_val),verbose=100, early_stopping_rounds=100,cat_features=cat_features)
            
            
            # pred=model.predict(X_val,prediction_type='RawFormulaVal')  # focal loss
            # coordinates=np.linspace(pred.min(),pred.max(),100)   # focal loss
            pred=model.predict_proba(X_val)[:,1]
            coordinates = np.linspace(0, 1, 100)

            best_score=0
            best_coordinate=0
            for coordinate in coordinates:
                pred_value=pred>coordinate
                score=f1_score(y_val,pred_value)
                if best_score<score:
                    best_score=score
                    best_coordinate=coordinate
            
            pred=(pred>best_coordinate)
            self.scores[iters].append(get_clf_eval(y_val,pred))
            self.thresholds[iters].append(best_coordinate)
            # scores.append(np.sqrt(mean_squared_error(y_val,pred)))
            self.models[iters].append(model)
        
        self.cv_scores.append(np.mean(self.scores[iters]))
        print(f'[{iters+1}] F1 scores mean: {self.cv_scores[iters]}')

    def modeling_kdata(self,n_estimators,max_depth,learning_rate,cat_features,train_datas,core='CPU'):
        for iter,(train_data,target) in enumerate(train_datas):
            self.modeling_kfold(iter,n_estimators,max_depth,learning_rate,cat_features,train_data,target,core=core)
        print(f'Total F1 scores mean: {np.mean(self.cv_scores)}')

    def predict(self,test_data):
        test_pred=pd.Series([0 for x in range(len(test_data))], index=test_data.index)
        for models,thresholds in zip(self.models,self.thresholds):
            for model,threshold in zip(models,thresholds):
                pred=model.predict_proba(test_data)[:,1]
                test_pred+=(pred>threshold)
        test_pred=test_pred/(self.k_data*self.k_fold)
        test_pred=test_pred.apply(lambda x:1 if x>0.5 else 0)
        return test_pred

In [27]:
cat_features=train_datas[0][0].columns.to_list()
cat_features.remove('bant_submit')
cat_features.remove('lead_desc_length')
cat_features.remove('historical_existing_cnt')
cat_features.remove('com_reg_ver_win_rate')
cat_features

['customer_country',
 'business_unit',
 'customer_idx',
 'customer_type',
 'enterprise',
 'customer_job',
 'inquiry_type',
 'product_category',
 'customer_position',
 'response_corporate',
 'expected_timeline',
 'business_area',
 'lead_owner',
 'strategic_ver',
 'region',
 'grant_weight']

In [28]:
kmodel=KMODEL(dataset_K=11)
kmodel.modeling_kdata(n_estimators=1000,max_depth=10,learning_rate=0.05,cat_features=cat_features,train_datas=train_datas)

-[1-1] fold-
0:	learn: 0.8402132	test: 0.8509804	best: 0.8509804 (0)	total: 109ms	remaining: 1m 49s
100:	learn: 0.9600921	test: 0.9271255	best: 0.9271255 (100)	total: 4.75s	remaining: 42.3s
200:	learn: 0.9838503	test: 0.9262837	best: 0.9283900 (182)	total: 8.71s	remaining: 34.6s
300:	learn: 0.9942122	test: 0.9306122	best: 0.9306122 (298)	total: 12.7s	remaining: 29.4s
400:	learn: 0.9981973	test: 0.9317023	best: 0.9317023 (398)	total: 16.6s	remaining: 24.8s
500:	learn: 0.9990985	test: 0.9317719	best: 0.9317719 (495)	total: 20.6s	remaining: 20.5s
600:	learn: 0.9998712	test: 0.9327902	best: 0.9327902 (592)	total: 24.6s	remaining: 16.3s
700:	learn: 1.0000000	test: 0.9316327	best: 0.9332654 (621)	total: 28.5s	remaining: 12.2s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9332654101
bestIteration = 621

Shrink model to first 622 iterations.
오차행렬:
 [[916  54]
 [ 77 913]]

정확도: 0.9332
정밀도: 0.9225
재현율: 0.9443
F1: 0.9333
-[1-2] fold-
0:	learn: 0.8354105	test: 0.8718210	best

0:	learn: 0.8351351	test: 0.8682171	best: 0.8682171 (0)	total: 39.6ms	remaining: 39.6s
100:	learn: 0.9619733	test: 0.9317719	best: 0.9328586 (85)	total: 3.78s	remaining: 33.7s
200:	learn: 0.9830203	test: 0.9323155	best: 0.9332654 (117)	total: 7.68s	remaining: 30.5s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9332654101
bestIteration = 117

Shrink model to first 118 iterations.
오차행렬:
 [[921  49]
 [ 78 912]]

정확도: 0.9352
정밀도: 0.9219
재현율: 0.9495
F1: 0.9355
[3] F1 scores mean: 0.9341882597940673
-[4-1] fold-
0:	learn: 0.8268063	test: 0.8414576	best: 0.8414576 (0)	total: 22.2ms	remaining: 22.2s
100:	learn: 0.9588130	test: 0.9342172	best: 0.9353031 (96)	total: 3.67s	remaining: 32.6s
200:	learn: 0.9816643	test: 0.9408163	best: 0.9413565 (181)	total: 7.4s	remaining: 29.4s
300:	learn: 0.9935683	test: 0.9403905	best: 0.9432805 (242)	total: 11.3s	remaining: 26.2s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9432805314
bestIteration = 242

Shrink mo

200:	learn: 0.9817762	test: 0.9360614	best: 0.9360614 (197)	total: 7.63s	remaining: 30.4s
300:	learn: 0.9925373	test: 0.9365404	best: 0.9384615 (280)	total: 11.5s	remaining: 26.8s
400:	learn: 0.9972935	test: 0.9376278	best: 0.9401535 (329)	total: 15.5s	remaining: 23.1s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9401534527
bestIteration = 329

Shrink model to first 330 iterations.
오차행렬:
 [[918  52]
 [ 66 924]]

정확도: 0.9398
정밀도: 0.9329
재현율: 0.9464
F1: 0.9396
[6] F1 scores mean: 0.9336439339895692
-[7-1] fold-
0:	learn: 0.8283136	test: 0.8521907	best: 0.8521907 (0)	total: 11ms	remaining: 11s
100:	learn: 0.9605449	test: 0.9309463	best: 0.9310873 (81)	total: 3.61s	remaining: 32.1s
200:	learn: 0.9841963	test: 0.9310700	best: 0.9343590 (150)	total: 7.49s	remaining: 29.8s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9343589744
bestIteration = 150

Shrink model to first 151 iterations.
오차행렬:
 [[911  59]
 [ 71 919]]

정확도: 0.9337
정밀도: 0.9277
재현율: 

100:	learn: 0.9584828	test: 0.9282443	best: 0.9293340 (65)	total: 3.67s	remaining: 32.7s
200:	learn: 0.9812628	test: 0.9283521	best: 0.9310873 (179)	total: 7.45s	remaining: 29.6s
300:	learn: 0.9942077	test: 0.9310169	best: 0.9324463 (240)	total: 11.3s	remaining: 26.3s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9324462641
bestIteration = 240

Shrink model to first 241 iterations.
오차행렬:
 [[919  51]
 [ 80 910]]

정확도: 0.9332
정밀도: 0.9199
재현율: 0.9474
F1: 0.9335
-[9-5] fold-
0:	learn: 0.8271115	test: 0.8450570	best: 0.8450570 (0)	total: 9.82ms	remaining: 9.81s
100:	learn: 0.9603093	test: 0.9242424	best: 0.9295489 (63)	total: 3.55s	remaining: 31.6s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.9295489103
bestIteration = 63

Shrink model to first 64 iterations.
오차행렬:
 [[916  54]
 [ 86 904]]

정확도: 0.9286
정밀도: 0.9142
재현율: 0.9443
F1: 0.9290
[9] F1 scores mean: 0.9319327483327756
-[10-1] fold-
0:	learn: 0.8332294	test: 0.8449460	best: 0.8449460 (0)	t

In [ ]:
pred=kmodel.predict(test_data)
pred.value_counts()